In [1]:
import json
import foursquareWithHosts
from datetime import datetime
from sys import stdout
with open('keys/appCredentialsSudo.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/accessTokensSudo.json') as tokensFile:    
    accessTokens = json.load(tokensFile)
    
# with open('keys/hosts.json') as hostsFile:    
#     hosts = json.load(hostsFile)

# Initialization

In [ ]:
clients = []
users = []
for accessToken in accessTokens[:5]:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

In [ ]:
auth_uri = client.oauth.auth_url()

In [ ]:
# Interrogate foursquare's servers to get the user's access_token
access_token = client.oauth.get_token('code')

# Apply the returned access token to the client
client.set_access_token(access_token)

# Get the user's data
user = client.users()

In [ ]:
client.set_access_token(accessTokens[1])

# Get the user's data
user = client.users()

# Get Activities

In [28]:
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [386]:
#Get current friends info
def getFriendsInfo(clients):
    usersFriends = []
    for client in clients[:5]:
        friendsInfo = client.users.friends()
        usersFriends.append(friendsInfo['friends'])
    return usersFriends

In [387]:
#Get current friends ids
def extractFriendsIds(usersFriends):
    avaliableFriends = []
    for userFriendSet in usersFriends:
        userFriendGroup = userFriendSet['items']
        friendIds = []
        for friend in userFriendGroup:
            friendIds.append(friend['id'])
        avaliableFriends.append(friendIds)
    return avaliableFriends

In [318]:
def countNewFriends(avaliableFriends):
    newFriends = 0
    for index, userSet in enumerate(avaliableFriends):
        for userId in userSet:
            if userId not in requestedActivitiesIds:
                newFriends = newFriends+1
    if newFriends:
        print('{} new friends').format(newFriends)
    else:
        print('No new friends')
    return newFriends

In [273]:
print len(requestedActivitiesIds)

71


In [321]:
#Get the activities of new users
def getFriendsActivities(avaliableFriends, clients, requestedActivitiesIds):
    userActivities = []

    for index, userSet in enumerate(avaliableFriends):
        progress(index, len(clients))
        for userId in userSet:
            if userId not in requestedActivitiesIds:
                userActivity = clients[index].users.activities(userId)
                userActivities.append(userActivity)
                db.activities.insert({'requestingUser':index,
                                        'requestedUserId':userId,
                                        'activity':userActivity,
                                        'requestedAt':datetime.now()})
                requestedActivitiesIds.append(userId)
    return requestedActivitiesIds

In [320]:
#Get all the data from DataBase
def getActivitiesFromDB(db):
    activityProjection = {"_id":0,
                          "activity.activities.items.content.object.venue.id":1,
                          "activity.activities.items.content.object.venue.name":1,
                            "activity.activities.items.content.object.venue.location.lat":1,
                            "activity.activities.items.content.object.venue.location.lng":1,
                            "activity.activities.items.content.object.checkin.venue.id":1,
                            "activity.activities.items.content.object.checkin.venue.name":1,
                             "activity.activities.items.content.object.checkin.venue.location.lat":1,
                             "activity.activities.items.content.object.checkin.venue.location.lng":1,
                             "activity.activities.items.createdAt":1,
                              "requestedUserId":1}
    activityQuery = {"activity.activities.items.content.type":"checkin"}
    usersVenues = db.activities.find(activityQuery, activityProjection)
    return usersVenues

In [322]:
#Transfer the data to python objects
def getActivitiesObjects(usersVenues):
    checkinsForUsers = []
    for userActivity in enumerate(usersVenues):
        checkinsForUser = []
        for checkIn in userActivity[1]['activity']['activities']['items']:
            checkinInfo = {}
            checkinInfo['userId'] = userActivity[1]['requestedUserId']
            checkinInfo["createdAt"] = checkIn['createdAt']
            checkinInfo["createdAtDate"] = str(extractComponents(checkIn['createdAt'], 240))#Get the real time zone

            if "venue" in checkIn['content']['object'].keys():
                checkinInfo["venueId"] = checkIn['content']['object']["venue"]["id"]
                checkinInfo["name"] = checkIn['content']['object']["venue"]["name"]
                checkinInfo["lat"] = checkIn['content']['object']["venue"]["location"]["lat"]
                checkinInfo["lng"] = checkIn['content']['object']["venue"]["location"]["lng"]
                checkinsForUser.append(checkinInfo)
            elif "venue" in checkIn['content']['object'].keys():
                checkinInfo["venueId"] = checkIn['content']['object']['checkin']["venue"]["id"]
                checkinInfo["name"] = checkIn['content']['object']['checkin']["venue"]["name"]
                checkinInfo["lat"] = checkIn['content']['object']['checkin']["venue"]["location"]["lat"]
                checkinInfo["lng"] = checkIn['content']['object']['checkin']["venue"]["location"]["lng"]
                checkinsForUser.append(checkinInfo)
        checkinsForUsers.append(checkinsForUser)
    return checkinsForUsers

In [440]:
def countTotalFriends(avaliableFriends):
    totalFriends = 0
    for userSet in avaliableFriends:
        totalFriends = totalFriends + len(userSet)
    print('{} friends total').format(totalFriends)

In [324]:
#Save data to tsv
def saveDataToCSV(checkinsForUsers):
    plainListOfCheckins = [item for sublist in checkinsForUsers for item in sublist]
    plainListOfCheckinsDF = pd.DataFrame(plainListOfCheckins)
    plainListOfCheckinsDF.to_csv('data/24June2/checkinsHistory.tsv', sep='\t',encoding='utf-8')

In [514]:
len(plainListOfCheckins)

10156

In [326]:
def saveRequestedActivitiesIds(requestedActivitiesIds):
    with open('data/requestedActivitiesIds.json','wb') as activitiesFile:
        json.dump(requestedActivitiesIds,activitiesFile)

In [ ]:
import requests
requests.packages.urllib3.disable_warnings()

In [441]:
def updateFriendListAndGetNewHistories(clients, requestedActivitiesIds):
    print datetime.now()
    #Check for new users
    usersFriends = getFriendsInfo(clients)
    avaliableFriends = extractFriendsIds(usersFriends)
    countTotalFriends(avaliableFriends)
    #Get activities of new ones
    if countNewFriends(avaliableFriends):
        print('Obtaining new users\' activities')
        requestedActivitiesIds = getFriendsActivities(avaliableFriends, clients, requestedActivitiesIds)
        usersVenues = getActivitiesFromDB(db)
        checkinsForUsers = getActivitiesObjects(usersVenues)
        saveDataToCSV(checkinsForUsers)
        saveRequestedActivitiesIds(requestedActivitiesIds)
    print ''

In [521]:
updateFriendListAndGetNewHistories(clients, requestedActivitiesIds)

2015-06-26 00:28:38.195373
118 friends total
No new friends



In [519]:
len(np.unique(requestedActivitiesIds))

201

# Run the loop

In [ ]:
import schedule
import time

schedule.clear()
schedule.every(2).minutes.do(updateFriendListAndGetNewHistories, clients, requestedActivitiesIds)

while True:
    schedule.run_pending()
    time.sleep(1)

# Some other scripts

In [474]:
(checkinsDF, checkpointsDF, usersDF) = fillSourceDataFrames('24June/')

In [479]:
len(np.unique(usersDF.userId))

833

In [ ]:
#4 Get the user that was checked-in
requestedUser = client.users('someUserId')

#5 Send requests to their friends 
if requestedUser['user']:
    requestedUserInfo = requestedUser['user']
    print 'Blocked:', requestedUserInfo['blockedStatus']
    print requestedUserInfo['homeCity']
    print requestedUserInfo['checkins']
    print requestedUserInfo['gender']
    print 'friends count:', requestedUserInfo['friends']['count']
    for friendGroup in requestedUserInfo['friends']['groups']:
        if friendGroup['type'] == 'friends':
            if friendGroup['count'] > 3:
                print 'Enough mutual friends'
                break
        if 0:
            print 'Already requested'
        print ''
        for friend in friendGroup['items']:
            if not friend['id'] in requestedUsersFriends:
                print 'Add'
                requestedUsersFriends.append(friend['id'])
                client.users.request(friend['id'])

In [422]:
#Request user Info
def getRequestedUsers(db):
    existingRequestedUsers = []
    existingRequestedUsersCoursor = db.requestedUsers.find()
    for requestedUser in existingRequestedUsersCoursor:
        existingRequestedUsers.append(int(requestedUser['requestedUser']['user']['id']))
    return existingRequestedUsers

def getUsersToRequest(usersDF, existingRequestedUsers):
    return usersDF[~usersDF.userId.isin(existingRequestedUsers)].userId.tolist()

def requestUsers(usersToRequest, db, clients):
    requestedUsers = []
    for index, userId in enumerate(usersToRequest):
        progress(index,len(usersToRequest))
        requestedUser = clients[index%len(clients)].users(userId)
        requestedUsers.append(requestedUser)
        db.requestedUsers.insert({'requestingUser':index%len(clients),
                                    'requestedUser':requestedUser,
                                     'requestedAt':datetime.now()})
        return requestedUsers

In [ ]:
#1 Get the venues of interes
yogaVenues = client.venues.search(params={'query': 'worldclass', 'near':'Moscow'})
for yogaPlace in yogaVenues['venues']:
    if yogaPlace['hereNow']['count'] > 0:
        print ''
        print yogaPlace['name']
        print yogaPlace['id']
        print 'here now:', yogaPlace['hereNow']['count']
        print 'total users:', yogaPlace['stats']['usersCount']